In [20]:
import duckdb
import pandas

In [59]:
df_games.columns

['column00',
 'Title',
 'Release Date',
 'Team',
 'Rating',
 'Times Listed',
 'Number of Reviews',
 'Genres',
 'Summary',
 'Reviews',
 'Plays',
 'Playing',
 'Backlogs',
 'Wishlist',
 'Plays_FLOAT']

In [60]:
duckdb.query("""    SELECT TITLE, SUM(PLAYS_FLOAT) as total_plays FROM 
                    df_games
                    group by title
                    order by total_plays desc
                    LIMIT 10
                  """)

┌─────────────────────────────────────────┬─────────────┐
│                  Title                  │ total_plays │
│                 varchar                 │   double    │
├─────────────────────────────────────────┼─────────────┤
│ Minecraft                               │    110000.0 │
│ The Legend of Zelda: Breath of the Wild │     90000.0 │
│ Grand Theft Auto V                      │     90000.0 │
│ Doom                                    │     89300.0 │
│ Portal 2                                │     87000.0 │
│ Undertale                               │     84000.0 │
│ Portal                                  │     84000.0 │
│ Among Us                                │     75000.0 │
│ Super Mario Odyssey                     │     75000.0 │
│ Super Smash Bros. Ultimate              │     75000.0 │
├─────────────────────────────────────────┴─────────────┤
│ 10 rows                                     2 columns │
└───────────────────────────────────────────────────────┘

In [ ]:
duckdb.query("""    SELECT \"Release Date\", TITLE, SUM(PLAYS_FLOAT) as total_plays FROM 
                    df_games
                    group by title
                    order by total_plays desc
                    LIMIT 10
                  """)